In [1]:
from fn import op, _
from itertools import repeat
from functools import reduce
# from objproxies import LazyProxy
import json
from copy import deepcopy, copy
from pipetools import pipe
from functools import partial

In [3]:
# def pipeline(*steps):
#     return reduce(lambda x, y: y(x), list(steps))
# def compose(*funcs):
#     return lambda x: reduce(lambda f, g: g(f), list(funcs), x)

In [4]:
# ToDo:
# Handle case where Mechanisms have no input. Perhaps the sentinel value of 0

In [5]:
# UI Behavior Mechanisms per Mechanism
def b1m1(s):
    return s['s1']
def b2m1(s):
    return s['s1'] * s['s2']

def b1m2(s):
    return s['s1']
def b2m2(s):
    return s['s1'] / s['s2']

# UI State Mechanisms per Mechanism
def s1m1(s, _input):
    s['s1'] = s['s1']**2 + _input
def s2m1(s, _input):
    s['s2'] = s['s2'] + 1 + _input  
def s3m1(s, _input):
    s['s3'] = s['s3']

def s1m2(s, _input):
    s['s1'] = s['s1'] + _input
def s2m2(s, _input):
    s['s2'] = s['s2']
def s3m2(s, _input):
    s['s3'] = s['s3'] + 1
    
j = {
    "m1": {
        "behaviors": {
            "b1": b1m1,
            "b2": b2m1
        },
        "states": {
            "s1": s1m1,
            "s2": s2m1,
            "s3": s3m1
        }
    },
    "m2": {
        "behaviors": {
            "b1": b1m2,
            "b2": b2m2
        },
        "states": {
            "s1": s1m2,
            "s2": s2m2,
            "s3": s3m2
        }
    }
}

In [6]:
def generate_configs(j):
    return list(
        map(
            lambda x: (
                list(j[x]["states"].values()),
                list(j[x]["behaviors"].values())
            ), 
            j.keys()
        )
    )

def getColResults(s, funcs):
    return list(map(lambda f: f(s), funcs))

def getBehaviorInput(s, funcs): 
    return op.foldr(_ + _)(getColResults(s, funcs))

def mech_step(sL, state_funcs, behavior_funcs):
    in_copy, out_copy, mutatable_copy = deepcopy(sL), deepcopy(sL), deepcopy(sL)
    last_in_obj, last_mut_obj = in_copy[-1], mutatable_copy[-1]
    
    _input = getBehaviorInput(last_in_obj, behavior_funcs)

    for f in state_funcs:
        f(last_mut_obj, _input)
    
    out_copy.append(last_mut_obj)
    return out_copy

def pipeline(states_list, configs):
    for config in configs:
        s_conf, b_conf = config[0], config[1]
        states_list = mech_step(states_list, s_conf, b_conf)
    return states_list

In [7]:
state_dict = {
    's1': 2,
    's2': 4,
    's3': 300
}
states_list = [state_dict]
configs = generate_configs(j)
pipeline(states_list, configs)

[{'s1': 2, 's2': 4, 's3': 300},
 {'s1': 14, 's2': 15, 's3': 300},
 {'s1': 28.933333333333334, 's2': 15, 's3': 301}]